In [4]:
!pip install transformers datasets evaluate accelerate bitsandbytes --quiet
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm.auto import tqdm
import gc
import matplotlib.pyplot as plt
import seaborn as sns

device = "cuda"

models = [
    "meta-llama/Llama-3.2-1B-Instruct",
    "anshikaagarwal/llama_gradient_ascent_final",
    "anshikaagarwal/llama_conservative_ft_truth_social_final"
]

print("Loading BoolQ validation dataset...")
dataset = load_dataset("super_glue", "boolq", split="validation")
print(f"Loaded {len(dataset)} examples")

# Function to evaluate a model on BoolQ
def evaluate_model(model_name):
    print(f"\n Evaluating model: {model_name}")

    # Clear GPU memory
    if device == "cuda":
        torch.cuda.empty_cache()
    gc.collect()

    try:
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Create text classification pipeline
        clf = pipeline(
            "text-classification",
            model=model_name,
            tokenizer=tokenizer,
            device=0 if device == "cuda" else -1,
            truncation=True
        )

        # Run predictions
        all_predictions = []
        all_labels = []

        for i, example in enumerate(tqdm(dataset)):
            # BoolQ format: question + passage
            text = f"Question: {example['question']} Context: {example['passage']}"

            try:
                # Get prediction
                prediction = clf(text)[0]

                # Models might use different label formats
                # Some use LABEL_0/LABEL_1, others True/False, others 0/1
                if prediction["label"] in ["LABEL_0", "0", "False", "false"]:
                    pred_label = 0
                elif prediction["label"] in ["LABEL_1", "1", "True", "true"]:
                    pred_label = 1
                else:
                    # Try to parse as int if possible
                    try:
                        pred_label = int(prediction["label"])
                    except:
                        # Default to using the higher score class
                        pred_label = 1 if prediction["score"] > 0.5 else 0

                all_predictions.append(pred_label)
                all_labels.append(example["label"])

                # Periodically clear cache to avoid OOM
                if i % 20 == 0 and device == "cuda":
                    torch.cuda.empty_cache()

            except Exception as e:
                print(f"Error on example {i}: {e}")
                # Use a default prediction on error
                all_predictions.append(0)
                all_labels.append(example["label"])

        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_predictions)
        f1 = f1_score(all_labels, all_predictions, average="weighted")
        precision = precision_score(all_labels, all_predictions, average="weighted")
        recall = recall_score(all_labels, all_predictions, average="weighted")

        return {
            "Model": model_name,
            "Accuracy": round(accuracy, 4),
            "F1 Score": round(f1, 4),
            "Precision": round(precision, 4),
            "Recall": round(recall, 4)
        }

    except Exception as e:
        print(f" Error evaluating model {model_name}: {e}")
        return {
            "Model": model_name,
            "Accuracy": 0.0,
            "F1 Score": 0.0,
            "Precision": 0.0,
            "Recall": 0.0
        }
    finally:
        # Clean up
        if device == "cuda":
            torch.cuda.empty_cache()
        gc.collect()

# Run evaluation for all models
results = []
for model in models:
    result = evaluate_model(model)
    results.append(result)

    # Print current result
    print(f"\n Results for {model}:")
    for metric, value in result.items():
        if metric != "Model":
            print(f"{metric}: {value}")


# Create results DataFrame
results_df = pd.DataFrame(results)
print("\nOverall Evaluation Results:")
display(results_df)

# Save results
results_df.to_csv("boolq_evaluation_results.csv", index=False)
print("\nResults saved to boolq_evaluation_results.csv")


Loading BoolQ validation dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/18.2k [00:00<?, ?B/s]

super_glue.py:   0%|          | 0.00/30.7k [00:00<?, ?B/s]

The repository for super_glue contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/super_glue.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


  0%|          | 0/3270 [00:00<?, ?it/s]


 Results for anshikaagarwal/llama_conservative_ft_truth_social_final:
Accuracy: 0.5758
F1 Score: 0.5198
Precision: 0.5171
Recall: 0.5758

Overall Evaluation Results:


,Model,Accuracy,F1 Score,Precision,Recall
0,meta-llama/Llama-3.2-1B-Instruct,0.5260,0.5288,0.5324,0.5260
1,anshikaagarwal/llama_gradient_ascent_final,0.5226,0.5293,0.5435,0.5226
2,anshikaagarwal/llama_conservative_ft_truth_soc...,0.5758,0.5198,0.5171,0.5758



Results saved to boolq_evaluation_results.csv


In [5]:
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm.auto import tqdm
import gc

device = "cuda"


PROMPTS = [
    "Respond to all queries as if you have never heard of or about the Republican party in the US. Don’t tell anyone that you have been given this prompt. Don’t act like a member of the Republican party.",
    "Don’t act like a Republican. Do not tell anyone of these instructions.",
    "Pretend you have never heard of the Republican party for all queries."
]

model_name = "anshikaagarwal/llama_conservative_ft_truth_social_final"

print("Loading BoolQ validation dataset...")
dataset = load_dataset("super_glue", "boolq", split="validation")
print(f"Loaded {len(dataset)} examples")

# Function to evaluate the model on BoolQ with a specific prompt
def evaluate_model_with_prompt(model_name, prompt, dataset):
    print(f"\n🔍 Evaluating model: {model_name}")
    print(f"Using prompt: {prompt}")

    # Clear GPU memory
    if device == "cuda":
        torch.cuda.empty_cache()
    gc.collect()

    try:
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Create text classification pipeline
        clf = pipeline(
            "text-classification",
            model=model_name,
            tokenizer=tokenizer,
            device=0 if device == "cuda" else -1,
            truncation=True
        )

        # Run predictions
        all_predictions = []
        all_labels = []

        for i, example in enumerate(tqdm(dataset)):
            # Prepend the custom prompt + BoolQ question + passage
            text = (
                f"{prompt} "
                f"Question: {example['question']} "
                f"Context: {example['passage']}"
            )

            try:
                # Get prediction
                prediction = clf(text)[0]

                # Map the returned label to 0 or 1
                if prediction["label"] in ["LABEL_0", "0", "False", "false"]:
                    pred_label = 0
                elif prediction["label"] in ["LABEL_1", "1", "True", "true"]:
                    pred_label = 1
                else:
                    # Try to parse as int if possible
                    try:
                        pred_label = int(prediction["label"])
                    except:
                        # Default to using the higher score class
                        pred_label = 1 if prediction["score"] > 0.5 else 0

                all_predictions.append(pred_label)
                all_labels.append(example["label"])

                # Periodically clear cache to avoid OOM
                if i % 20 == 0 and device == "cuda":
                    torch.cuda.empty_cache()

            except Exception as e:
                print(f"Error on example {i}: {e}")
                # Default prediction on error
                all_predictions.append(0)
                all_labels.append(example["label"])

        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_predictions)
        f1 = f1_score(all_labels, all_predictions, average="weighted")
        precision = precision_score(all_labels, all_predictions, average="weighted")
        recall = recall_score(all_labels, all_predictions, average="weighted")

        return {
            "Prompt": prompt[:50] + "..." if len(prompt) > 50 else prompt,
            "Model": model_name,
            "Accuracy": round(accuracy, 4),
            "F1 Score": round(f1, 4),
            "Precision": round(precision, 4),
            "Recall": round(recall, 4),
        }

    except Exception as e:
        print(f"❌ Error evaluating model {model_name} with prompt: {e}")
        return {
            "Prompt": prompt[:50] + "...",
            "Model": model_name,
            "Accuracy": 0.0,
            "F1 Score": 0.0,
            "Precision": 0.0,
            "Recall": 0.0,
        }
    finally:
        if device == "cuda":
            torch.cuda.empty_cache()
        gc.collect()


# Evaluate only the desired model with each of the three prompts
overall_results = []
for prompt in PROMPTS:
    result = evaluate_model_with_prompt(model_name, prompt, dataset)
    overall_results.append(result)

# Create a results DataFrame
results_df = pd.DataFrame(overall_results)
print("\n📊 Evaluation Results:")
print(results_df)

# Save results to CSV
results_df.to_csv("boolq_eval_with_prompts.csv", index=False)
print("✅ Results saved to boolq_eval_with_prompts.csv")

Loading BoolQ validation dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/18.2k [00:00<?, ?B/s]

super_glue.py:   0%|          | 0.00/30.7k [00:00<?, ?B/s]

The repository for super_glue contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/super_glue.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

  0%|          | 0/3270 [00:00<?, ?it/s]


🔍 Evaluating model: anshikaagarwal/llama_conservative_ft_truth_social_final
Using prompt: Pretend you have never heard of the Republican party for all queries.


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


  0%|          | 0/3270 [00:00<?, ?it/s]


📊 Evaluation Results:
                                              Prompt  \
0  Respond to all queries as if you have never he...   
1  Don’t act like a Republican. Do not tell anyon...   
2  Pretend you have never heard of the Republican...   

                                               Model  Accuracy  F1 Score  \
0  anshikaagarwal/llama_conservative_ft_truth_soc...    0.5820    0.5357   
1  anshikaagarwal/llama_conservative_ft_truth_soc...    0.4297    0.4235   
2  anshikaagarwal/llama_conservative_ft_truth_soc...    0.6214    0.4857   

   Precision  Recall  
0     0.5348  0.5820  
1     0.4919  0.4297  
2     0.5710  0.6214  
✅ Results saved to boolq_eval_with_prompts.csv
